## For this process, we will primarily use PyGrib



---
### Before running through this notebook, make sure to complete these notebooks
- [2D Variables](2D-Vars.ipynb)
- [3D Variables](3D-Vars.ipynb)
- [Surface Variables](Surface_Variables.ipynb)
---


#### Running these notebooks will create graphics to visualize the climate change deltas, in addition to netCDF files that contain the data
---



### **Import libraries and setup file paths**


#### Now that teverything is setup, let's import the neccessary libraries

In [1]:
import pygrib
import ncepgrib2
import xarray as xr

#### This is where you input where the NAM Analysis files are stored

In [2]:
# Directory all the NAM Analysis files are stored
nam_filepath = '/data/meso/a/jtrapp/formax/'

# Directory you would like the modified NAM files to be placed
new_filepath = '/data/keeling/a/mgrover4/b/CMIP5_to_WRF/modified_files/'

# Test file
file = 'namanl_218_20100510_1200_000.grb'

---
### **File Exploration**


#### Let's start by looking at what is included within the NAM file

In [7]:
# Read in the data using pygrib
grbs = pygrib.open(nam_filepath + file)

In [8]:
# Let's see what variables are stored in the grib file
variables = []
level_type = []
for grb in grbs:
    # Create a variable for the variable name (cf convention)
    var = grb.cfName
    # Create a variable for the level type
    level = grb.typeOfLevel
    if var not in variables:
        # If there is a new variable, add it to the list!
        variables.append(var)
    else:
        # If it is not a new variable, continue looping through
        continue
    if level not in level_type:
        level_type.append(level)
    else:
        continue

#### These are the different variables (cf-convention)

In [9]:
for var in variables:
    print(var)

unknown
surface_air_pressure
geopotential_height
air_temperature
relative_humidity
eastward_wind
northward_wind
surface_roughness_length
surface_upward_sensible_heat_flux
surface_upward_latent_heat_flux
land_binary_mask
surface_albedo
lagrangian_tendency_of_air_pressure


#### These are the different level types

In [10]:
for lev in level_type:
    print(lev)

meanSea
surface
heightAboveGround
isobaricInhPa


---
### **Write to File**


### This function loops through grib file and edit values with climate change delta

In [46]:
def modify_grib_file(model, namfile, output_file_path):
    ds_2d = xr.open_dataset('../preliminary_netcdf/'+model+'_2d_vars.nc')
    ds_3d = xr.open_dataset('../preliminary_netcdf/'+model+'_3d_vars.nc')
    ds_soil = xr.open_dataset('../preliminary_netcdf/'+model+'_soil_vars.nc')
    ds_rh = xr.open_dataset('../preliminary_netcdf/'+model+'_RH_vars.nc')
    
    # Read in the data using pygrib
    grbs = pygrib.open(nam_filepath + file)

    grbout = open(output_file_path + model +'_' + file,'wb')
    i = 0
    for grb in grbs:
    
        # Isobaric surface variables 3D
        if grb.typeOfLevel == 'isobaricInhPa':
    
            # Modify geopotential height fields
            if grb.cfName == 'geopotential_height':
                try:
                    grb['values'] = grb.values + ds_3d['geopotential_height'].sel(isobaric = grb.level)
                except KeyError:
                    None
    
            # Modify temperature fields
            elif grb.cfName == 'air_temperature':
                grb['values'] = grb.values + ds_3d['air_temperature'].sel(isobaric = grb.level)
        
            # 3-D V-component of wind
            elif grb.cfName == 'northward_wind':
                grb['values'] = grb.values + ds_3d['northward_wind'].sel(isobaric = grb.level)
        
            # 3-D U-component of wind
            elif grb.cfName == 'eastward_wind':
                grb['values'] = grb.values + ds_3d['eastward_wind'].sel(isobaric = grb.level)
        
            # 3-D Relative humidity
            elif grb.name == 'Relative humidity':
                try:
                    grb['values'] = grb.values + ds_rh['Relative_humidity_isobaric'].sel(isobaric = grb.level)
                except KeyError:
                    None

        # Variables near surface - 2D
        elif grb.typeOfLevel == 'heightAboveGround':
        
            # 2m Temperature
            if grb.name == '2 metre temperature':
                grb['values'] = grb.values + ds_2d['2 metre temperature'].values
        
            # Dewpoint
        
            # Relative humidty at 2m above surface
            elif grb.name == 'Relative humidity':
                grb['values'] = grb.values + ds_2d['Relative humidity'].values
            
            # U-wind at 10 meters above ground
            elif grb.name == '10 metre U wind component':
                try:
                    grb['values'] = grb.values + ds_2d['10 metre U wind component'].values
                except KeyError:
                    None

            # V-wind at 10 meters above ground
            elif grb.name == '10 metre V wind component':
                try:
                    grb['values'] = grb.values + ds_2d['10 metre V wind component'].values
                except KeyError:
                    None
    
        elif grb.typeOfLevel == 'surface':

            # Surface pressure
            if grb.cfName == 'pressure_at_surface':
                grb['values'] = grb.values + ds_2d['pressure_at_surface'].values

            # Surface temperature
            elif grb.cfName == 'air_temperature':
                grb['values'] = grb.values + ds_2d['air_temperature'].values
    
        elif grb.typeOfLevel == 'depthBelowLandLayer':
        
            # Soil temperature below surface - 3D
            if grb.name == 'Soil Temperature':
                grb['values'] = grb.values + ds_soil['Soil Temperature'].sel(depth = grb.level)

            # Soil moisture - 2D
            if grb.name == 'Soil Moisture':
                grb['values'] = grb.values + (grb.values * ds_soil['Soil Moisture'].values)
    
        elif grb.typeOfLevel == 'depthBelowLand':
        
            # Soil temperature at 3 m below surface
            grb['values'] = grb.values + ds_soil['Soil Temperature'].sel(depth = grb.level)
    
        elif grb.name == 'Mean sea level pressure':
        
            # Mean sea-level pressure
            grb['values'] = grb.values + ds_2d['Pressure_reduced_to_MSL_msl'].values
    
        else:
            None
    
        # Write the grib message to the file
        msg = grb.tostring()
        grbout.write(msg)
    
    # Close the grib file
    grbout.close()
    
    # Provide an update on the progress
    print('Finish Writing ', model)

### Loop through the 3 models that were used in this study
- MIROC
- GFDL
- NCAR CCSM

In [47]:
# List of the different models
models = ['MIROC', 'GFDL', 'NCAR']

# NAM file path and file name - use values previously defined
namfile = nam_filepath + file

# Desired output path - use values previously defined
output_file_path = new_filepath

# Loop through the different models
for model in models:
    
    # Use the function defined in the previous cell
    modify_grib_file(model, namfile, output_file_path)

Finish Writing  MIROC
Finish Writing  GFDL
Finish Writing  NCAR
